In [7]:
import argparse
import cv2
import os
import numpy as np
from tqdm import tqdm
import re
import imutils
from resolutions import resolution_label_map, read_info

In [8]:
def write_next_img(filename, txtname):
    f = open(txtname, "a")
    f.write("\n"+filename+" ")
    f.close()
    
def write_rect(rect):
    global TXT
    f = open(TXT, "a")
    f.write(np.array_str(rect)+" ")
    f.close()

def click_and_crop(event, x, y, flags, param):
    global rect, tip_x, tip_y, clone
    h,w,_ = clone.shape
    if event == cv2.EVENT_LBUTTONDOWN:
        rect=np.concatenate((rect,[x,y]))
        if len(rect)==4:
            write_rect(rect)
            rect=np.array([])
    elif event == cv2.EVENT_MOUSEMOVE:
        white=(255,255,255)
        clone  = image.copy()
        if len(rect)==2:
            cv2.rectangle(clone, tuple(map(int,rect)), (x,y), (0,255,0))
        cv2.line(clone, (0,y),(w,y),white)
        cv2.line(clone, (x,0),(x,h),white)
        cv2.imshow("image", clone)

# Set scene, paths and read resolution

In [9]:
SCENE = "dark_1"

DIR = "/home/maciej/Desktop/pallet_dataset/scenes/"+SCENE+"/"
TXT = "/home/maciej/Desktop/pallet_dataset/labels/"+SCENE+".txt"

_,_,_,RESOLUTION,LABEL_RESOLUTION=read_info(DIR)

# Label

In [10]:
filenames = sorted(os.listdir(DIR))
filenames=[f for f in filenames if f != "info.txt"]
#filenames=["r_2_99.jpg"]
cv2.namedWindow("image")
cv2.setMouseCallback("image", click_and_crop)

#variables
rect = np.array([])
tip_x, tip_y = (0,0)

for i in tqdm(range(len(filenames))):
    filename=filenames[i]
    print(filename)
    path=DIR+filename
    image = cv2.imread(path)
    #image = imutils.rotate(image, 1.5)
    if RESOLUTION!=LABEL_RESOLUTION:
        image = cv2.resize(image, LABEL_RESOLUTION)
    clone = image.copy()
    write_next_img(filename, TXT)
    cv2.imshow("image", clone)
    cv2.waitKey(0)
    rect = np.array([])
cv2.destroyAllWindows()

  0%|          | 0/5 [00:00<?, ?it/s]

IMG_20200228_103440.jpg


 20%|██        | 1/5 [00:01<00:05,  1.38s/it]

IMG_20200228_103442.jpg
IMG_20200228_103443.jpg


 80%|████████  | 4/5 [00:01<00:00,  1.31it/s]

IMG_20200228_103445.jpg
IMG_20200228_103446.jpg


100%|██████████| 5/5 [00:02<00:00,  2.40it/s]


# Display labelled

In [11]:
def parse_line(line):
    filename,points=line.split(" ", 1)
    point_list=points.split("] [")
    point_list=[points.strip('][ ').replace('.','').split(' ') for points in point_list]
    point_list=[[int(point) for point in points if point] for points in point_list]
    point_list=[(tuple(points[:2]),tuple(points[2:4])) for points in point_list]
    point_list=[((0,0),(0,0)) if not points[0] and not points[1] else points for points in point_list]
    print(point_list)
    return filename,point_list

with open(TXT) as f:
    lines = [line.rstrip('\n') for line in f]
lines = [parse_line(line) for line in lines]
cv2.namedWindow("image")
for line in lines:
    filename,point_list=line
    path=DIR+filename
    image = cv2.imread(path)
    if RESOLUTION!=LABEL_RESOLUTION:
        image = cv2.resize(image, LABEL_RESOLUTION) 
    for points in point_list:
        p1,p2=points
        cv2.rectangle(image, p1, p2, (0,255,0))
    cv2.imshow("image", image)
    cv2.waitKey(0)
cv2.destroyAllWindows()

[((220, 303), (783, 405))]
[((239, 329), (798, 432))]
[((237, 205), (807, 312))]
[((239, 406), (801, 512))]
[((241, 347), (804, 453))]
